# Trade Tracking Simulation

 - 10분에 한번씩 거래 check



#### 01. 매도 조건

 1. 예측 고가 도달시
 
 
 2. 손절 요건 미충족시 그날 15:20 종가로 매매


#### 02. 손절 조건 (아래 요건 중 1개만 달성해도 매도)

 1. 매수가격(시가)를 '0'으로 놓고 예측가격을 '1'로 놓았을 때 10번의 체크 기간 동안 0.3을 단 한번도 못넘었을 때
 
 
 2. 10번의 check 기간동안 8번이 전 기간 종가가 하락할 때
 
 
 3. check 1과 check 10의 차이가 오차범위 이내일 때
 
 
 4. 이전 check 단계 대비 종가가 10% 이상 하락이 1번이라도 나타날 시
 
  
 이 중 2개 이상을 만족할시 손절

#### 03. 종목 셋팅

In [1]:
code = '035760'
pred = 96926             
date = '20200318'

#### 04. 시뮬레이션

In [2]:
# 경고 무시
import warnings
warnings.filterwarnings(action='ignore') 

# Hyper Parameter
tr_name = 'TR_SCHART'
term = '1'
start_date = date
end_date = date
Lookup = '9999' 
        
# url
url = 'http://ssecd.roboadvisor.co.kr:9999/' + tr_name + '?0=' + code + '&1=' + term + '&2=10' +\
      '&3=' + start_date + '&4=' + end_date + '&5=' + Lookup

# url open
from urllib.request import urlopen
url_page = urlopen(url)

# json 파일로 받아오기
import json
url_data = json.loads(url_page.read())

# json을 데이터프레임으로 바꾸기
import pandas as pd
data = pd.DataFrame(url_data)
    
# Data Column명 변경
data.columns = ['date','time','open','high','low','close','price_ccr','volume_ccr',
                'rock','volume','volume_price']
data = data[['date', 'time', 'open', 'high', 'low', 'close', 'volume', 'volume_price']]
    
# 순서 뒤집기
data = data[::-1]
data['date_time'] = data['date'] + '_' + data['time']
data = data.set_index('date_time')
    
# 필요한 변수만 추출
data = data[['open', 'high', 'low', 'close', 'volume']]

# 변수 속성 정수로 바꾸기
for column in data.columns : 
    data[column] = data[column].apply(lambda x : int(x))

# 진행상태 표시
data['state'] = ''

# Simulation
simulation_df = pd.DataFrame()

for idx in data.index : 
    
    # 시가에 매수
    if idx == date + '_0910' : 
        temp = data.loc[idx]
        temp['state'] = '매수'
        simulation_df = simulation_df.append(temp)
        continue
        
    # 고가가 목표치에 도달시 종가로 매도
    if data.loc[idx]['high'] > pred : 
        temp = data.loc[idx]
        temp['state'] = '매도'
        simulation_df = simulation_df.append(temp)
        break
        
    # 15:20 종가로 일괄 매도
    if idx == date + '_1510' : 
        temp = data.loc[idx]
        temp['state'] = '매도'
        simulation_df = simulation_df.append(temp)
        break
    
    # 손절 요건이 충족되지 않을 경우 계속 보유
    temp = data.loc[idx]
    temp['state'] = '보유'
    simulation_df = simulation_df.append(temp)
    
# 피쳐 순서
simulation_df = simulation_df[['open', 'high', 'low', 'close', 'volume', 'state']]

#### 05. 수익률

In [3]:
buy = simulation_df.loc['20200318_0910']['open']
cell = int(simulation_df[-1:]['close'])
rate = (100 * (cell - buy)) / buy
print('종목 코드 : {},     매수가 : {},     매도가 : {},     수익률 : {:.2f}%' .format(code, buy, cell, rate))

종목 코드 : 035760,     매수가 : 94600.0,     매도가 : 96600,     수익률 : 2.11%


#### 06. 시뮬레이션 결과

In [4]:
simulation_df

,open,high,low,close,volume,state
20200318_0910,94600.0,95000.0,93000.0,93700.0,12553.0,매수
20200318_0920,93600.0,94300.0,93100.0,93200.0,6052.0,보유
20200318_0930,93300.0,94100.0,92700.0,93800.0,6324.0,보유
20200318_0940,93900.0,93900.0,92700.0,93000.0,3674.0,보유
20200318_0950,93000.0,93300.0,92600.0,92600.0,4496.0,보유
20200318_1000,92700.0,92700.0,92200.0,92300.0,3713.0,보유
20200318_1010,92300.0,92900.0,92200.0,92500.0,3351.0,보유
20200318_1020,92600.0,92600.0,92300.0,92300.0,3095.0,보유
20200318_1030,92500.0,93000.0,92400.0,92900.0,1998.0,보유
20200318_1040,93000.0,93500.0,92600.0,93100.0,1660.0,보유
